In [ ]:
# tests/test_rest.py
import os
import tempfile
import pytest
from fastapi.testclient import TestClient
from app.main import app
from app.database import Base, engine, SessionLocal
from app import models

client = TestClient(app)

@pytest.fixture(scope="module", autouse=True)
def setup_db():
    # Use the same sqlite for simplicity; ensure DB is created
    Base.metadata.create_all(bind=engine)
    # create a sample bank and branch if none present
    db = SessionLocal()
    if not db.query(models.Bank).first():
        bank = models.Bank(name="Test Bank")
        db.add(bank); db.commit(); db.refresh(bank)
        branch = models.Branch(ifsc="TEST0001", branch="Main", city="Hyderabad", state="Telangana", bank_id_fk=bank.id)
        db.add(branch); db.commit()
    db.close()
    yield
    # no teardown for now

def test_get_banks():
    res = client.get("/banks")
    assert res.status_code == 200
    data = res.json()
    assert isinstance(data, list)
    assert any(b.get("name") == "Test Bank" for b in data)

def test_get_branch_by_ifsc():
    res = client.get("/branches/TEST0001")
    assert res.status_code == 200
    data = res.json()
    assert data["ifsc"] == "TEST0001"
    assert data["bank"]["name"] == "Test Bank"